<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [10]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 71.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [11]:
%load_ext sql

In [12]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [13]:
!pip install -q pandas==1.1.5

In [14]:
%sql sqlite:///my_data1.db

In [15]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


**Note:This below code is added to remove blank rows from table**


In [16]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [18]:
%%sql
select Launch_Site
from SPACEXTABLE 
group by Launch_Site;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [19]:
%%sql
select * 
from SPACEXTABLE 
where Launch_Site like 'CCA%' 
limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [20]:
%%sql
select Customer, sum(PAYLOAD_MASS__KG_) as TotalPayloadMass 
from SPACEXTABLE 
where Customer like 'NASA (CRS)' 
group by Customer;

 * sqlite:///my_data1.db
Done.


Customer,TotalPayloadMass
NASA (CRS),45596.0


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [21]:
%%sql
select Booster_Version, AVG(PAYLOAD_MASS__KG_) as AvePayloadMass 
from SPACEXTABLE 
where Booster_Version like 'F9 v1.1' 
group by Booster_Version;

 * sqlite:///my_data1.db
Done.


Booster_Version,AvePayloadMass
F9 v1.1,2928.4


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [22]:
%%sql 
select Landing_Outcome, Date as FirstSuccess 
from SPACEXTABLE 
where Landing_Outcome like 'Success (ground pad)' 
group by Landing_Outcome 
order by substr(Date,7)||substr(Date,4,2)||substr(Date,1,2) asc
limit 1;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,FirstSuccess
Success (ground pad),22/12/2015


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [23]:
%%sql 
select Booster_Version 
from SPACEXTABLE 
where PAYLOAD_MASS__KG_ between 4000 and 6000 
and Landing_Outcome like 'Success (drone ship)'
group by Booster_Version;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1021.2
F9 FT B1031.2
F9 FT B1022
F9 FT B1026


### Task 7




##### List the total number of successful and failure mission outcomes


In [25]:
%%sql 
select ltrim(rtrim(Mission_Outcome)), count(*) 
from SPACEXTABLE
group by ltrim(rtrim(Mission_Outcome));

 * sqlite:///my_data1.db
Done.


ltrim(rtrim(Mission_Outcome)),count(*)
Failure (in flight),1
Success,99
Success (payload status unclear),1


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [29]:
%%sql
select Booster_Version
from SPACEXTABLE a
where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE)
group by Booster_Version
order by Booster_Version;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1048.5
F9 B5 B1049.4
F9 B5 B1049.5
F9 B5 B1049.7
F9 B5 B1051.3
F9 B5 B1051.4
F9 B5 B1051.6
F9 B5 B1056.4
F9 B5 B1058.3


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.**


In [34]:
%%sql
select substr(Date, 4, 2) as month2015, Landing_Outcome, Booster_Version, Launch_Site
from SPACEXTABLE
where substr(Date,7,4)='2015'
and Landing_Outcome like 'Failure (drone ship)'
order by substr(Date, 4, 2);

 * sqlite:///my_data1.db
Done.


month2015,Landing_Outcome,Booster_Version,Launch_Site
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [42]:
%%sql
select Landing_Outcome, count(*) as CNTLandingOutcomes
from SPACEXTABLE
where cast(substr(Date,7)||substr(Date,4,2)||substr(Date,1,2) as int) between 20100604 and 20170320
group by Landing_Outcome
order by count(*) desc;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,CNTLandingOutcomes
No attempt,10
Success (ground pad),5
Success (drone ship),5
Failure (drone ship),5
Controlled (ocean),3
Uncontrolled (ocean),2
Precluded (drone ship),1
Failure (parachute),1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
